# Predicting House Sale Prices

In this project, we'll explore ways to build and improve a linear regression model by working with housing data for the city of Ames, Iowa from 2006 to 2010. Information on the dataset can be found [here](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627), and the columns info can be found [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

## Building the Pipeline

We'll start by importing our libraries, reading in our data, and then setting up a pipeline of functions that will help us quickly iterate over different models.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

pd.options.display.max_columns = 999

In [2]:
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [3]:
# Returns training Data Frame
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse


transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

print("RMSE: ", round(rmse, 2))

RMSE:  57088.25


## Feature Engineering

Next, we'll begin removing features that have too many missing values, look for potential categorical features, and transform the text and numerical columns.

We'll start by updating our `transform_features()` function so that any column with more than XX% missing values is dropped. We'll also remove columns that leak information about the sale. We'll need to read the data documentation to get a better idea of what transformations are necessary and for which columns.

1) For **all columns**, drop any with 5% or more missing values:

In [8]:
# Create series with missing values data
val_missing = df.isnull().sum()

# Filter the columns containing > 5% missing values
missing_val_cols = val_missing[(val_missing > len(df)/20)].sort_values()

# Drop missing value columns
df = df.drop(missing_val_cols.index, axis=1)

2) For **text columns**, drop any with 1 or more missing values:

In [9]:
# Create series with missing text values data
text_missing = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Filter the text columns containing any missing values
missing_text_cols = text_missing[text_missing > 0]

# Drop missing value columns
df = df.drop(missing_text_cols.index, axis=1)

3) For **numerical columns**, fill in with the most common value in that column:

In [10]:
# Compute missing value counts
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_num_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
fixable_num_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [11]:
# Compute the most common value for each of the columns in `fixable_num_cols`
replacement_vals = df[fixable_num_cols.index].mode().to_dict(orient='records')[0]
replacement_vals

{'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 0.0,
 'Garage Cars': 2.0,
 'Garage Area': 0.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [12]:
# Replace missing values
df = df.fillna(replacement_vals)

# Verify no missing values left
df.isnull().sum().value_counts()

0    64
dtype: int64

Next, let's look at what new features we can create to better capture some of the information in the dataset.

In [17]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [18]:
years_remodel = df['Yr Sold'] - df['Year Remod/Add']
years_remodel[years_remodel < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [19]:
# Make new columns
df['Years Before Sale'] = years_sold
df['Years Since Remodel'] = years_remodel

# Drop the rows with negative values for both new features
df = df.drop([1702, 2180, 2181], axis=0)

# Remove original year columns
df = df.drop(['Year Built', 'Year Remod/Add'], axis = 1)

Next, we'll drop the columns that aren't useful in machine learning models and columns that leak data about the final sale.

In [20]:
# Drop columns that are not useful
df = df.drop(['PID', 'Order'], axis=1)

# Drop columns that leak info
df = df.drop(['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)

Now let's update the `transform_features()` function.

In [21]:
def transform_features(df):
    
    val_missing = df.isnull().sum()
    missing_val_cols = val_missing[(val_missing > len(df)/20)].sort_values()
    df = df.drop(missing_val_cols.index, axis=1)
    
    text_missing = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    missing_text_cols = text_missing[text_missing > 0]
    df = df.drop(missing_text_cols.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_num_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_vals = df[fixable_num_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_vals)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_remodel = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remodel'] = years_remodel
    df = df.drop([1702, 2180, 2181], axis=0)
    
    df = df.drop(['Year Built', 'Year Remod/Add', 'PID', 'Order', 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis = 1)
    
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

print("RMSE: ", round(rmse, 2))

RMSE:  55275.37


## Feature Selection

Now that we've cleaned and transformed our data, we can move on to selecting our features for our model.

We'll start by looking at how numerical features in our training set correlate with `SalePrice`.

In [23]:
numerical_df = transform_df.select_dtypes(include=['int', 'float'])
numerical_df.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remodel
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,2.0,528.0,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1.0,730.0,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1.0,312.0,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,2.0,522.0,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,2.0,482.0,212,34,0,0,0,0,0,189900,13,12


In [26]:
corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values(ascending=False)
corr_coeffs

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648361
Total Bsmt SF          0.644012
Garage Area            0.641425
1st Flr SF             0.635185
Years Before Sale      0.558979
Full Bath              0.546118
Years Since Remodel    0.534985
Mas Vnr Area           0.506983
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.439284
Wood Deck SF           0.328183
Open Porch SF          0.316262
Half Bath              0.284871
Bsmt Full Bath         0.276258
2nd Flr SF             0.269601
Lot Area               0.267520
Bsmt Unf SF            0.182751
Bedroom AbvGr          0.143916
Enclosed Porch         0.128685
Kitchen AbvGr          0.119760
Screen Porch           0.112280
Overall Cond           0.101540
MS SubClass            0.085128
Pool Area              0.068438
Low Qual Fin SF        0.037629
Bsmt Half Bath         0.035875
3Ssn Porch             0.032268
Misc Val

In [27]:
# Keep only the columns with a correlation coefficient > 0.4
corr_coeffs[corr_coeffs > 0.4]

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648361
Total Bsmt SF          0.644012
Garage Area            0.641425
1st Flr SF             0.635185
Years Before Sale      0.558979
Full Bath              0.546118
Years Since Remodel    0.534985
Mas Vnr Area           0.506983
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.439284
Name: SalePrice, dtype: float64

In [28]:
# Drop the columns with a correlation coefficient < 0.4
transform_df = transform_df.drop(corr_coeffs[corr_coeffs < 0.4].index, axis=1)

Next, let's decide which categorical columns we should keep. Some columns may be numerical but need to be encoded as categorical instead, and some columns with too many unique values might be best left out, otherwise we'd have to add hundreds of dummy variable columns to our dataframe.

In [29]:
# List of column names that are supposed to be categorical
nominal_features = ['PID', 'MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 'Lot Config', 'Neighborhood', 
                    'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 
                    'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 
                    'Misc Feature', 'Sale Type', 'Sale Condition']

In [ ]:


DQ:
## Which categorical columns have we still carried with us? We'll test tehse 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

## How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [ ]:

Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:
If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).
Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
What are some ways we can explore which categorical columns "correlate" well with SalePrice?
Read this post for some potential strategies.
Update the logic for the select_features() function. This function should take in the new, modified train and test data frames that were returned from transform_features().

## Train & Test

## Conclusion & Next Steps

In this project we ...

The idea for this project comes from the [DATAQUEST](https://app.dataquest.io/) **Linear Regression For Machine Learning** course.